# Part 1:

Modeling Challenge
#Python Coding and Data Set



In [ ]:
#Load in the data file and header file provided
import pandas as pd
import csv

with open("field_names.txt","r") as f:
     field_names= f.read().splitlines()
with open('breast-cancer.csv',newline='') as f:
    r = csv.reader(f)
    data = [line for line in r]
with open('breast-cancer.csv','w',newline='') as f:
    w = csv.writer(f)
    w.writerow(field_names)
    w.writerows(data)

In [ ]:
#The dataframe does not currently have a header, load in the header file and attach it to the dataframe

data = pd.read_csv('breast-cancer.csv')
print(data.columns)

In [ ]:
#Compute the mean and median smoothness and compactness for benign and malignant tumors - do they differ? Explain how you would identify this.
# Mean smoothness for Mallignant cells
print('Mean smoothness for Mallignant cells -',np.mean(data[data['diagnosis']=='M']['smoothness_mean']))

# Median smoothnes for Mallignant cells
print('Median smoothness for Mallignant cells -', np.median(data[data['diagnosis']=='M']['smoothness_mean']))

# Mean smoothness for Benign cells
print('Mean smoothness for Benign cells -',np.mean(data[data['diagnosis']=='B']['smoothness_mean']))

# Median smoothnes for Benign cells
print('Median smoothnes for Benign cells -',np.median(data[data['diagnosis']=='B']['smoothness_mean']))

# Mean compactness for Mallignant cells
print('Mean compactness for Mallignant cells -',np.mean(data[data['diagnosis']=='M']['compactness_mean']))

# Median compactness for Mallignant cells
print('Median compactness for Mallignant cells -', np.median(data[data['diagnosis']=='M']['compactness_mean']))

# Mean compactness for Benign cells
print('Mean compactness for Benign cells -',np.mean(data[data['diagnosis']=='B']['compactness_mean']))

# Median compactness for Benign cells
print('Median compactness for Benign cells -',np.median(data[data['diagnosis']=='B']['compactness_mean']))

In [ ]:
#Write a function to generate bootstrap samples of the data.

def sample_data(size, dataset):
    sample_data = dataset.values[np.random.randint(size, size=size)]
    return sample_data

#example
s_data = sample_data(30,data)

print('Size of sampled data',s_data.shape)

In [ ]:
#Exploratory Analysis



In [ ]:
#Identify 2-3 variables that are predictive of a malignant tumor.
#Display the relationship visually and write 1-2 sentences explaining the relationship.
# encoding 'diagnosis' feature as 0/1

from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
le.fit(data['diagnosis'])
data['diagnosis']=le.transform(data['diagnosis'])

# print top 4 correlation coefficients
for i in sorted(corr_numeric["diagnosis"])[-5:]:
    if i != 1.0:
        print(corr_numeric[corr_numeric["diagnosis"] == i].index.values[0])
import seaborn as sns
sns.set(style="ticks")
sns.pairplot(data, x_vars=['diagnosis'], y_vars=['perimeter_sd_error','concavity_worst','concave_points_sd_error','fractal_dimension_mean'])

import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20,15))
corr_numeric = data.corr()

sns.heatmap(corr_numeric, cbar=True, cmap="RdBu_r")
plt.title("Correlation Matrix", fontsize=18)
plt.show()

In [ ]:
#Modeling
#Build a model to predict the malignant tumors.
#Use at least two classification techniques; compare and contrast the advantages and disadvantages of each.
#Identify how you would control for overfitting in each classification technique.
#Evaluate the performance of each model.
#In each model, identify the most important predictive variables and explain how you identified them.

In [ ]:

X = data.loc[: , data.columns != 'diagnosis']
print(X.shape)

Y = data['diagnosis']
print(Y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=5)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# checking uniform distribution of y labels
print(y_train.value_counts(), y_test.value_counts())


# as we had seen there is strong correlations between predicted variables and target variables. 
# Logistic Reg won't help us solving this problem
# dataset violates LR assumptions

from sklearn.linear_model import LogisticRegression

#random state to allow reproducibility of results
LR_model= LogisticRegression(random_state=0)
LR_model.fit(X_train, y_train)

y_pred = LR_model.predict(X_test)

from sklearn.metrics import *

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


# checking if model overfits

y_pred_train = LR_model.predict(X_train)

print(accuracy_score(y_train, y_pred_train))
print(confusion_matrix(y_train, y_pred_train))

In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=4, random_state=0)
clf.fit(X_train, y_train)

y_train_pred = clf.predict(X_train)
print('Train acc -',accuracy_score(y_train, y_train_pred))
print('Train conf matrix \n',confusion_matrix(y_train, y_train_pred), '\n-----')

# Testing 
y_pred = clf.predict(X_test)
print('Train acc -', accuracy_score(y_test,y_pred))
# test accuracy < train accuracy (good to go)
print('Train conf matrix \n', confusion_matrix(y_test, y_pred), '\n-----')

# Control for overfitting
# Cross validation performed for more rigorous analysis (check against overfitting)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, X, Y, cv=3)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# checking top 3 highest contributing variables

ind_top = clf.feature_importances_.argsort()[-3:][::-1]
print('Top 3 contributing variables are -- ')
for i in ind_top:
    print(X_train.columns[i])

# Note - this result is line with our EDA above for correaltion diagram
# Top contributing varibales are same 

In [ ]:
#GBM
from sklearn.ensemble import GradientBoostingClassifier

GBC_clf = GradientBoostingClassifier(random_state=0)
GBC_clf.fit(X_train,y_train)
y_train_pred = GBC_clf.predict(X_train)
print('Training acc -',accuracy_score(y_train,y_train_pred))
print('Train conf matrix \n',confusion_matrix(y_train,y_train_pred), '\n-----')

# no false positives/negatives -> indicating overfitting
# additionally 1 classificaito score also indicating same

y_test_pred = GBC_clf.predict(X_test)
print('Test acc -',accuracy_score(y_test, y_test_pred))
print('Test conf matrix \n',confusion_matrix(y_test, y_test_pred), '\n-----')

# Control for overfitting
# Cross validation performed for more rigorous analysis (check against overfitting)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(GBC_clf, X, Y, cv=4)
print("CV Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# checking top 3 highest contributing variables

ind_top = GBC_clf.feature_importances_.argsort()[-3:][::-1]
print('Top 3 contributing variables are -- ')
for i in ind_top:
    print(X_train.columns[i])
    
# pretty much inline with our above analysis

# Explanation
To Technical Audiences

Explain the limitations of your analysis and identify possible further steps you could take.

However please understand that the model should be further developed ( this is a short version) but in normal circumstances I would have used Tensorflow or PyTorch using a neural network and not simply Random Forest or GBM. However for these limited purposes these should do ( for now). On a TF network a similar model I built obtained 97% accuracy.

There is also a need for the correlation factors as well as F score . Accuracy might be misleading in this case to use as a single performance criteria. ROC might came into play as well. 


To Non-Technical Audiences


The dataset consists of 569 medical observations with 212 Malign datarows so the data set is balanced and needs no augmentation or recalibration. 

The dataset altough of certain quality ( no NaN ) is headerless so we had to apply the heading corresponding to each feature and to the label.

We identified 4 major features which are driving a high corelation to the target diagnosis:perimeter_sd_error','concavity_worst','concave_points_sd_error','fractal_dimension_mean'. These have shown a large coreelation to the diagnosis. 

This modelling is binary(benign /malign) therefore as first step of our analysis the accuracy score was used for both models: over 90% result. 
 The dataset is small in nature and prone to overfitting ( i.e. following too close the diagnosis). This is a major limitation of the analysis and bigger dataset should be provided. 
 

Write a short summary of your analysis, explaining how your model works and how it performs.

The two models 

Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean/average prediction (regression) of the individual trees.[1][2] Random decision forests correct for decision trees' habit of overfitting to their training set.[3]:587–588 Random forests generally outperform decision trees, but their accuracy is lower than gradient boosted trees. However, data characteristics can affect their performance.[4]

Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees.

Briefly explain the factors that contributed to malignant vs benign tumor identification.

# Part 2

# student-sample-1.py
#!/usr/bin/env python

import pandas as pd
import numpy as np

#from sklearn import LinearRegression
from sklearn.linear_model import LinearRegression
#from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score

# Load data
d = pd.read_csv('../data/train.csv')


# Setup data for prediction

# incorrect reference of dataframe "data". Correct reference is 'd'
#x1 = data.SalaryNormalized
x1 = d.SalaryNormalized
#same here
#x2 = pd.get_dummies(data.ContractType)
y = pd.get_dummies(d.ContractType)
#you should use y for labels ( x for features)

# Setup model
model = LinearRegression()

# Evaluate model. Your model is completely empty. 
# Please re-access your model requirements i.e.
# what is it that you want to predict i.e. target variable (Y)
# identify your inputs i.e. predictor variables (X)
# using above variable for your model is a bad design choice. Will give very poor results.

# incorrect import statement 
from sklearn.model_selection import cross_val_score
#from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

# using cv=1, beats the purpose of k-fold cross validation
# when cv=1, it effectively becomes a simple test-train split.
# Which doesn't help in reducing overfitting

# Some changes in attribute order for cross_val_score
# The correct format for cross_val_score is cross_val_score(estimator, X, y=None....)
# updating attribute names x1=>x , x2=>Y
# 'scoring' parameter doesn't define the type of error metric i.e. incorrect attribute value

#scores = cross_val_score(model, x2, x1, cv=1, scoring='mean_absolute_error')

## correct format --
scores = cross_val_score(model, x1, y, cv=5)

print(scores.mean())

# student-sample-2.py

#!/usr/bin/env python

import pandas as pd
import numpy as np
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
#from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import cross_val_score

# Load data
data = pd.read_csv('../data/train.csv')


# Setup data for prediction
y = data.SalaryNormalized
X = pd.get_dummies(data.ContractType)

# Setup model
model = LinearRegression()
# Evaluate model

# Please re-access your model requirements i.e.
# what is it that you want to predict i.e. target variable (Y)
# identify your inputs i.e. predictor variables (X)
# using above variable for your model is a bad design choice. Will give very poor results.

# Some changes in attribute order for cross_val_score
# The correct format for cross_val_score is cross_val_score(estimator, X, y=None....)
# updating attribute names x1=>x , x2=>Y
# 'scoring' parameter doesn't define the type of error metric i.e. incorrect attribute value

# Evaluate model
scores = cross_val_score(model, X, y, cv=5, scoring='mean_absolute_error')
# you can read more about cross_val_score on sklearn website https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

## correct format --
## scores = cross_val_score(model, x1, y, cv=5)
print(scores.mean())